* [참고] 필사 원문 https://www.kaggle.com/code/drgilermo/dynamics-of-new-york-city-animation/notebook#Winter-vs-Summer
* [참고] import OO와 from OO import*의 차이 https://coding-kindergarten.tistory.com/73
* [참고] 파이썬 날짜 다루기(datetime, dateutil, timedelta) https://jsp-dev.tistory.com/entry/Python-%EB%82%A0%EC%A7%9C-%EB%8B%A4%EB%A3%A8%EA%B8%B0-datetime-dateutil-timedelta
* [참고] 머신러닝, 딥러닝 이상치(outlier) 데이터 탐지(detection) 및 제거하기 https://lsjsj92.tistory.com/556
* [참고] Jupyter Notebook 셀에서 값을 연속적으로 모두 출력하려면?https://financedata.github.io/posts/display-all-values-in-cell-jupyter-notebook.html
* [참고] AttributeError: 'Series' object has no attribute 'reshape'
https://stackoverflow.com/questions/53723928/attributeerror-series-object-has-no-attribute-reshape
* [참고] TypeError: 'numpy.ndarray' object is not callable
https://itsmycode.com/solved-numpy-ndarray-object-is-not-callable-python/


## Data fields
* id - a unique identifier for each trip
* vendor_id - a code indicating the provider associated with the trip record
* pickup_datetime - date and time when the meter was engaged
* dropoff_datetime - date and time when the meter was disengaged
* passenger_count - the number of passengers in the vehicle (driver entered value)
* pickup_longitude - the longitude where the meter was engaged
* pickup_latitude - the latitude where the meter was engaged
* dropoff_longitude - the longitude where the meter was disengaged
* dropoff_latitude - the latitude where the meter was disengaged
* store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip 
### <-- What does it mean?
* trip_duration - duration of the trip in seconds
Disclaimer: The decision was made to not remove dropoff coordinates from the dataset order to provide an expanded set of variables to use in Kernels. 

How does the traffic of Taxi rides change along the day?

In order to answer this question, I would use K-means clustering to cluster New York into different groups based on location, **and analyze the traffic into and out of every cluster as a function of the time along the day.** <-- 다시 읽어보기  One can expect that residential areas would have more incoming traffic in the evening, whereas commercial areas would mostly attract people during the day, and areas with rich nightlife would show more traffic in the night.

This can be helpful in duration prediction, as we can learn what are the likely destinations for each area during different times of the day.  <-- 다시 읽어보기  

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib.pyplot import * # matplotlb.pyplot 모듈로부터 모두(*)
# import 모듈 -> import만 사용하면 모듈 안의 함수를 사용할 때, 모듈명.함수명()으로 하고,
# from 모듈 import * -> from을 사용하면 바로 함수명()으로 사용
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm    # colormap
from sklearn.cluster import KMeans    # K-평균 군집 알고리즘
from sklearn.neighbors import KNeighborsClassifier    #
from dateutil import parser    # dateutil은 datetime 모듈의 확장 라이브러리
# parse를 사용하면 날짜형식을 대충 넣어도 datetime.datetime객체로 잘 바꾸어준다!
import io    # 파이썬3에서 파일 입출력(input/output)을 제공하는 모듈
import base64    # base64 방식으로 인코딩해주는 모듈(base64는 이진숫자들을 64개의 ASCII를 사용하여 일련의 문자열로 변환하는 인코딩 방식)
from IPython.display import HTML    # DataFrame()을 HTML로 출력    # Jupyter Notebook 셀에서 마지막 값 하나만이 아니라 모든 값을 차례로 출력
from imblearn.under_sampling import RandomUnderSampler    
from subprocess import check_output    
print(check_output(["ls", "../input"]).decode("utf8"))    
import warnings
warnings.filterwarnings('ignore')

### Read the data

In [2]:
df = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip')

In [3]:
df.head()

### Remove rides to and from far away areas (이상치 제거)
이상치(outlier) = 일반적인 데이터 패턴과 다르게 매우 이상한 패턴을 가지고 있는 등의 데이터
왜 제거? 이상치 데이터가 모델의 성능을 크게 좌우한다.
왜냐하면 모델의 입장에서는 데이터의 패턴을 보아야하는데 이러한 이상치 데이터들 때문에 좀 뜬금없는 패턴을 패턴을 학습하게 되기 때문이다.

In [4]:
xlim = [-74.03, -73.77]
ylim = [40.64, 40.85]
df = df[(df.pickup_longitude> xlim[0]) & (df.pickup_longitude < xlim[1])]
# 이 범위는 어떻게 찾았을까? -> https://www.google.com/maps/place/40%C2%B037'48.0%22N+74%C2%B001'48.0%22W/@40.6966776,-74.0998859,10.21z/data=!4m5!3m4!1s0x0:0x1a6533bb80c3e89e!8m2!3d40.63!4d-74.03
# https://www.google.com/maps/place/40%C2%B051'00.0%22N+73%C2%B046'12.0%22W/@40.7330052,-74.1669282,9.94z/data=!4m5!3m4!1s0x0:0x75ecf10492ac949c!8m2!3d40.85!4d-73.77
# NYC https://www.google.com/maps/place/%EB%AF%B8%EA%B5%AD+%EB%89%B4%EC%9A%95/@40.7670935,-74.0744074,10.5z/data=!4m5!3m4!1s0x89c24fa5d33f083b:0xc80b8f06e177fe62!8m2!3d40.7127753!4d-74.0059728
df = df[(df.dropoff_longitude> xlim[0]) & (df.dropoff_longitude < xlim[1])]
df = df[(df.pickup_latitude> ylim[0]) & (df.pickup_latitude < ylim[1])]
df = df[(df.dropoff_latitude> ylim[0]) & (df.dropoff_latitude < ylim[1])]

### Plot rides
미터기가 켜진 지점, 꺼진 지점 / 위도 x 경도
=> 교통량

In [5]:
longitude = list(df.pickup_longitude) + list(df.dropoff_longitude)
latitude = list(df.pickup_latitude) + list(df.dropoff_latitude)
plt.figure(figsize = (10, 10))
plt.plot(longitude, latitude, '.', alpha = 0.4, markersize = 0.05) # point marker
# alpha가 투명도니까 1이 되면 안보여야 되는 게 아닌가...?
plt.show()

In [6]:
loc_df = pd.DataFrame()
loc_df['longitude'] = longitude
loc_df['latitude'] = latitude

### Cluster(군집)
Let's cluster New York City based on the pick-up and drop-off points of each taxi ride

[참고]

In [7]:
kmeans = KMeans(n_clusters=15, random_state=2, n_init = 10).fit(loc_df)
# 15개의 클러스터를 찾도록 지정
# -> 알고리즘이 클러스터 중심(cluster center)으로 삼을 데이터 포인트 10개를 무작위로 초기화(n_init)
# 그런 다음 알고리즘이 반복됨

loc_df['label'] = kmeans.labels_ # labels_속성은 각 row가 어떤 그룹에 속하는지 그 결과를 표현
print(loc_df['label'])


In [8]:
loc_df.head(5)

In [9]:
# 그룹핑 결과를 시각화
loc_df = loc_df.sample(200000) # sample()메소드는 DataFrame으로부터 랜덤으로 표본을 추출(random sampling)
plt.figure(figsize = (10, 10))
for label in loc_df.label.unique():
    plt.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label], '.', alpha = 0.3, markersize =0.3)
plt.title('Clusters of New York')
plt.show()   

As we can see, the clustering results in a partition which is somewhat similar to the way NY is divided into different neighborhoods. We can see Upper East and West side of Central park in gray and pink respectively. West midtown in blue, Chelsea and West Village in brown, downtown area in blue, East Village and SoHo in purple.

The airports JFK and La LaGuardia have there own cluster, and so do Queens and Harlem. Brooklyn is divided into 2 clusters, and the Bronx has too few rides to be separated from Harlem.

Let's plot the cluster centers:

In [10]:
fig, ax = plt.subplots(figsize = (10,10)) # fig = 도화지, ax = 그림 하나하나, plt.subplots() 그래프 한 번에 여러개 그리기
for label in loc_df.label.unique():
    # 밑그림
    ax.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label], '.', alpha = 0.4, markersize = 0.1, color = 'gray')
    
    # 점찍기
    ax.plot(kmeans.cluster_centers_[label,0], kmeans.cluster_centers_[label,1],'o', color = 'r')
    
    # 주석달기 - 중심점 위치들
    ax.annotate(label, (kmeans.cluster_centers_[label,0], kmeans.cluster_centers_[label, 1]), color = 'b', fontsize = 20)
    # <-- kmeans.cluster_centers_[label, 0또는 1은 무엇?]  => 위도, 경도
ax.set_title('Cluster Centers')
plt.show()

In [11]:
df['pickup_cluster'] = kmeans.predict(df[['pickup_longitude', 'pickup_latitude']]) 
# 훈련 세트에 대해 predict() 메서드를 실행하면 labels_ 와 같은 결과를 얻습니다.
# 즉, 여기서는 predict() 메서드를 통해 unlabeled 데이터를 그룹에 할당합니다.
df['dropoff_cluster'] = kmeans.predict(df[['dropoff_longitude','dropoff_latitude']])
df['pickup_hour'] = df.pickup_datetime.apply(lambda x: parser.parse(x).hour)
# 내가 만든 커스텀 함수를 DataFrame에 적용하려면 apply 함수를 사용

In [12]:
df.head(5)

In [13]:
clusters = pd.DataFrame()
clusters['x'] = kmeans.cluster_centers_[:,0] 
clusters['y'] = kmeans.cluster_centers_[:,1]
clusters['label'] = range(len(clusters))

In [14]:
clusters.head(5)

In [15]:
loc_df = loc_df.sample(5000)    # sample()메소드는 DataFrame으로부터 랜덤으로 표본을 추출(random sampling)

In [16]:
loc_df.head(5)

### Taxi rides from one cluster to another(클러스터 사이에 일어난 택시 주행)
And the following animation, every arrow represents rides from one cluster to another. The width of the arrow is proportional to the relative amount of trips in the relevant hour.

In [17]:
fig, ax = plt.subplots(1, 1, figsize = (10,10))    
# plt.subplots()는 Matplotlib의 Figure 객체와 Axes객체를 반환
# Axes 객체는 축(Axis), 눈금(Tick), 그래프 곡선(Line 2D), 텍스트(Text), 다각형(Polygon) 등 그래프의 다양한 구성 요소 등을 포함하는 객체

# animation 돌릴 준비
def animate(hour):
    ax.clear()    # 다음에 그림을 그리는 명령이 실행될 때, 그 전 그림을 지웁니다.
    ax.set_title('Absolute Traffic - Hour ' + str(int(hour)) + ':00')
    plt.figure(figsize = (10,10));
    for label in loc_df.label.unique():
        ax.plot(loc_df.longitude[loc_df.label == label], loc_df.latitude[loc_df.label == label], '.', alpha = 1, markersize = 2, color = 'gray');
        ax.plot(kmeans.cluster_centers_[label,0],kmeans.cluster_centers_[label, 1], 'o', color = 'r');
        
        for label in clusters.label:
            for dest_label in clusters.label:
                num_of_rides = len(df[(df.pickup_cluster == label) & (df.dropoff_cluster == dest_label) & (df.pickup_hour == hour)])
                dist_x = clusters.x[clusters.label == label].values[0] - clusters.x[clusters.label == dest_label].values[0]
                dist_y = clusters.y[clusters.label == label].values[0] - clusters.y[clusters.label == dest_label].values[0]
                pct = np.true_divide(num_of_rides,len(df))    # / 연산자는 np.true_divide() 함수의 약어였음 - 요소 별 분할. 한 배열을 다른 배열로 나누고 결과를 세 번째 배열에 저장
                arr = Arrow(clusters.x[clusters.label == label].values, clusters.y[clusters.label == label].values, -dist_x, -dist_y, edgecolor='white', width = 15*pct)
                ax.add_patch(arr)    # matplotlib.patches 모듈은 그래프에 표현할 수 있는 다양한 도형 클래스를 포함하고 있다.
                                     # add_patch()를 써서 patches 모듈의 도형들을 그래프 영역에 추가
                arr.set_facecolor('g')    # axes 객체에 배경색 설정 - green
                                               

                    
# animation 돌리기
ani = animation.FuncAnimation(fig, animate, sorted(df.pickup_hour.unique()), interval = 1000)    
# matplotlib.animation 모듈의 FuncAnimation 클래스는 지정한 함수를 반복적으로 호출함으로써 애니메이션을 생성함
# FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128))
# FuncAnimation 클래스의 첫번째 인자로 Figure 객체를, 두번째 인자로는 프레임마다 호출할 함수를 지정합니다.
# frames의 값은 updata 함수의 반복변수로 들어갑니다.
# interval로 프레임 사이의 간격을 밀리초(milliseconds) 단위로 지정 (default = 200)

plt.close()    # figure창 닫기(figure를 여러 개 띄울 때, 창을 닫아야만 그 다음 코드로 진행이 됨)

ani.save('animation.gif', writer='imagemagick', fps=2) 
# matplotlib.animation모듈의 ArtistAnimation 클래스의 save() 메서드는 모든 프레임을 애니메이션으로 저장함
# Matplotlib 애니메이션을 저장하기 위해 ImageMagick을 사용합니다.
filename = 'animation.gif'
video = io.open(filename, 'r+b').read()    
# open()함수는 file을 불러옴 - open('파일 주소 및 이름', 출력 모드,'w 또는 r 또는 a')
# r+b mode? r+ is used for reading, anad writing mode. b is for binary. r+b mode is open the binary file in read or wirte mode.
# 파일입출력의 형태에는 w(write, 쓰기), r(read, 읽기), a(add, 추가하기) 세 가지가 있다.
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii'))) # DataFrame()을 HTML로 출력    
# Jupyter Notebook 셀에서 마지막 값 하나만이 아니라 모든 값을 차례로 출력
                                        

In [18]:
# relative traffic은 absolute와 어떤 부분에 다르게 만들어졌을까?

fig, ax = plt.subplots(1, 1, figsize = (10,10))

def animate(hour):
    ax.clear()
    ax.set_title('Relative Traffic - Hour ' + str(int(hour)) + ':00')    
    plt.figure(figsize = (10,10))
    for label in loc_df.label.unique():
        ax.plot(loc_df.longitude[loc_df.label == label],loc_df.latitude[loc_df.label == label],'.', alpha = 1, markersize = 2, color = 'gray')
        ax.plot(kmeans.cluster_centers_[label,0],kmeans.cluster_centers_[label,1],'o', color = 'r')


    for label in clusters.label:
        for dest_label in clusters.label:
            num_of_rides = len(df[(df.pickup_cluster == label) & (df.dropoff_cluster == dest_label) & (df.pickup_hour == hour)])
            dist_x = clusters.x[clusters.label == label].values[0] - clusters.x[clusters.label == dest_label].values[0]
            dist_y = clusters.y[clusters.label == label].values[0] - clusters.y[clusters.label == dest_label].values[0]
            pct = np.true_divide(num_of_rides,len(df[df.pickup_hour == hour]))
            arr = Arrow(clusters.x[clusters.label == label].values, clusters.y[clusters.label == label].values, -dist_x, -dist_y, edgecolor='white', width = pct)
            ax.add_patch(arr)
            arr.set_facecolor('g')


ani = animation.FuncAnimation(fig,animate,sorted(df.pickup_hour.unique()), interval = 1000)
plt.close()
ani.save('animation.gif', writer='imagemagick', fps=2)
filename = 'animation.gif'
video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

We can see that in the morning most of the traffic is in Manhattan island.
The share of taxis travelling to the Brookly area, mostly Williamsburg, becomes much larger in the late evening. Since there's no similar movement in the morning hours (in the opposite direction), this is unlikely to be the result of commuting. Instead, and since the traffic is mostly seen after 22:00, there are probably people going out.

since the arrows represent the relative traffic in the relevant hour, it is also possible that the increasing width of the arrows leading to Brooklyn may simply be a result of the reduction in the rides in Manhattan, due to the commercial character of big parts of it. But when looking at the absolute traffic, the arrows from Manhattan to Brooklyn are barely seen for the most part of the day.

In the very early morning, most of the traffic is to and from the two airports. As we can learn from the absolute graph, this is merely the result of decrease in traffic in the parts of town.

### Neighborhood Analysis
Let's manually assign the nighborhood name to each cluster

In [19]:
neighborhood = {-74.0019368351: 'Chelsea',-73.837549761: 'Queens',-73.7854240738: 'JFK',-73.9810421975:'Midtown-North-West',-73.9862336241: 'East Village',
                -73.971273324:'Midtown-North-East',-73.9866739677: 'Brooklyn-parkslope',-73.8690098118: 'LaGuardia',-73.9890572967:'Midtown',-74.0081765545: 'Downtown'
                ,-73.9213024854: 'Queens-Astoria',-73.9470256923: 'Harlem',-73.9555565018: 'Uppe East Side',
               -73.9453487097: 'Brooklyn-Williamsburgt',-73.9745967889:'Upper West Side'}

# 경도값으로 군집에 이름을 할당하려고 하나보다

In [20]:
rides_df = pd.DataFrame(columns = neighborhood.values())
rides_df['name'] = neighborhood.values()

rides_df.head(5)

In [21]:
# k-NN 알고리즘 (k-최근접 이웃 분류)
# 가장 간단한 머신러닝 알고리즘으로, 훈련 데이터셋을 그냥 저장하는 것이 모델을 만드는 과정의 전부이다.
# KNeighborsClassifier에서의 학습은 새로운 데이터 포인트에 대해 예측할 때
# 알고리즘이 훈련 데이터셋에서 가장 가까운 데이터 포인트, 즉 '최근접 이웃'을 찾을 수 있도록
# 데이터를 저장하는 것이다.
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(np.array(list(neighborhood.keys())).reshape(-1, 1), list(neighborhood.values())) # 모델 훈련 fit(X_train, y_train) 

In [22]:
# predict 메서드를 호출해서 모델이 예측해낸 값을 데이터프레임에 저장 
df['pickup_neighborhood'] = neigh.predict(df.pickup_longitude.values.reshape(-1,1))
df['dropoff_neighborhood'] = neigh.predict(df.dropoff_longitude.values.reshape(-1,1))

# neigh.predict(df.pickup_longitude.reshape(-1,1)) 
# -> [AttributeError]'Series' object has no attribute 'reshape'
# => 1차원 배열을 2차원 배열로 만들어 주면 된다.
# X.values.reshape(-1, 1)
# Y.values.reshape(-1, 1)


# 클러스터별로 지역이름을 담고있던 ride_df에 새 열을 추가 - 출발로 기록된 지점과 모델이 예측한 도착 지점이 같은 경우
for col in rides_df.columns[:-1]:
    rides_df[col] = rides_df.name.apply(lambda x: len(df[(df.pickup_neighborhood == x) & (df.dropoff_neighborhood == col)]))

In [23]:
df.head(5)

In [24]:
rides_df.head(5)

Now let's plot a heatmap and look where do passengers go to and from (These are aggregate values of all rides):

In [25]:
import plotly.offline as py    # 주피터 노트북에서 오프라인으로 플로팅할 때
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)    # 주피터 노트북에서 사용하기 위해 - 노트북 세션의 시작 부분에서 노트북 모드를 켠다


'''
trace = go.Heatmap(z = np.array(rides_df.values()),    # pandas dataframe 을 numpy 배열로 변환하는 as_matrix 사라짐 -> values
                   x = rides_df.columns[:-1],
                   y = rides_df.columns)
layout = dict(
    title = ' <b>Neighborhoods Interaction</b>',
    titlefont = dict(
    size = 30,
    color = ('rgb(100,100,100)')),
    margin = dict(t=100, r=100, b=100, l=150),
        yaxis = dict(
            title = ' <b> From </b>'),
        xaxis = dict(
            title = ' <b> To </b>'))
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')    # 주피터 노트북에서 플롯틀리 그래프를 그릴때는 iplot 
'''

In [26]:
# matplotlib.pyplot 모듈의 matshow() 함수를 이용해서 2차원 어레이 형태의 숫자 데이터 -> 히트맵 그리기
fig,ax = plt.subplots(figsize = (12,12))
cax = ax.matshow(rides_df.drop('name',axis = 1),interpolation='nearest',cmap=cm.afmhot)
cbar = fig.colorbar(cax)
ax.grid('off')
ax.set_xticks(range(len(rides_df)))
ax.set_xticklabels(rides_df.name, rotation =90,fontsize = 15)
ax.set_yticks(range(len(rides_df)))
ax.set_yticklabels(rides_df.name,fontsize = 15)
ax.set_xlabel('To', fontsize = 25)
ax.set_ylabel('From', fontsize = 25)
ax.set_title('Neighborhoods Interaction', y=1.35, fontsize = 30)

In [27]:
rides_df.index = rides_df.name
rides_df = rides_df.drop('name', axis =1)
rides_df.head(5)

In [28]:
rides_df.shape

We can see that center Manhattan neighborhoods are the most taxi-crowded, with Upper East side as the leader. We also see that the most common ride is within the cluster(which can not be seen in the animation).

The heatmap is fairy symmetrical, meaning that there aren't clusters that significantly have more pickups than dropoffs or the otehr way aronund. Let's zoom on it:

heatmap만 가지고는 확신이 조금 부족

In [29]:
#### 선그래프 그리기
fig, ax = plt.subplots(figsize = (12, 12))
for i in range(len(rides_df)):
    ax.plot(rides_df.sum(axis = 1)[i],rides_df.sum(axis = 0)[i],'o', color = 'b')
    ax.annotate(rides_df.index.tolist()[i], (rides_df.sum(axis = 1)[i],rides_df.sum(axis = 0)[i]), color = 'b', fontsize = 12)
    
ax.plot([0,250000],[0,250000], color = 'r', linewidth = 1)
ax.grid('off')    # <-- off를 줬는데 격자무늬가 표시되네?
ax.set_xlim([0,250000])
ax.set_ylim([0,250000])
ax.set_xlabel('Outbound Taxis')
ax.set_ylabel('Inbound Taxis')
ax.set_title('Inbound and Outbound rides for each cluster')

We can see that the inbound-outbound ratio for each neighborhood is relatively balanced.

The two airports have more outbound rides than inbound, which makes sense - drivers would probably go to the airport even without passengers, to have the chance to take people into the city. The residential area - Queens, Brookly and Harlem have more inbound ride, whereas the more commercial and touristic areas have more outbound.  with Upper East and West, being both commercial and residential, almost on the curve.

It seems that people would go into Manhattan by alternative means of transportation, but are more likely to get out of it by a cab.